# STA 141B: Homework 6
Winter 2018

## Student Information

After the colons (in the same line) please write just your first name, last name, and the 9 digit student ID number below.

First Name: Zihao

Last Name: Chen

Student ID: 915490404

## Instructions

We use a script that extracts your answers by looking for cells in between the cells containing the exercise statements.  So you 

- MUST add cells in between the exercise statements and add answers within them and
- MUST NOT modify the existing cells, particularly not the problem statement

To make markdown, please switch the cell type to markdown (from code) - you can hit 'm' when you are in command mode - and use the markdown language.  For a brief tutorial see: https://daringfireball.net/projects/markdown/syntax

# Web scraping the Aggie

In this assignment, you'll scrape text from [The California Aggie](https://theaggie.org/) and then analyze the text.

The Aggie is organized by category into article lists. For example, there's a [Campus News](https://theaggie.org/campus/) list, [Arts & Culture](https://theaggie.org/arts/) list, and [Sports](https://theaggie.org/sports/) list. Notice that each list has multiple pages, with a maximum of 15 articles per page.

The goal of exercises 1.1 - 1.3 is to scrape articles from the Aggie for analysis in exercise 1.4.

__Exercise 1.1.__ Write a function that extracts all of the links to articles in an Aggie article list. The function should:

* Have a parameter `url` for the URL of the article list.

* Have a parameter `page` for the number of pages to fetch links from. The default should be `1`.

* Return a list of aricle URLs (each URL should be a string).

Test your function on 2-3 different categories to make sure it works.

Hints:

* Be polite to The Aggie and save time by setting up [requests_cache](https://pypi.python.org/pypi/requests-cache) before you write your function.

* Start by getting your function to work for just 1 page. Once that works, have your function call itself to get additional pages.

* You can use [lxml.html](http://lxml.de/lxmlhtml.html) or [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to scrape HTML. Choose one and use it throughout the entire assignment.

In [1]:
import lxml.html as lx
import lxml.html as lx
import pandas as pd
import requests
import requests_cache
import time

requests_cache.install_cache("cache")

In [2]:
def Articles_url(url,page=1):
    """
    The function is to extracts all of the links to articles in an Aggie article list
    
    Arg:
    url:the url like https://theaggie.org/campus/, should end with /
    page: how many does the pages be attracted, the default is to attract only the first page.
    """
    article=[]
    for i in range(1,page+1):
        # Step 1: download the page.
        link=str(url)+"page/"+str(i)
        response = requests.get(link)
        doc = response.text
        
        # Step 2: parse
        html = lx.fromstring(doc, base_url = link)
        html.make_links_absolute()
        article.extend([x.attrib['href'] for x in html.xpath("//h2[@class = 'entry-title']/a")])

    return article

In [3]:
Articles_url("https://theaggie.org/campus/")#to check the default 'page' para.

['https://theaggie.org/2018/03/13/last-week-in-senate-25/',
 'https://theaggie.org/2018/03/12/uc-regents-will-vote-on-nonresident-tuition-hike-on-wednesday/',
 'https://theaggie.org/2018/03/12/cheating-allegations-in-soc-46a/',
 'https://theaggie.org/2018/03/12/chancellor-creates-three-task-forces-to-improve-student-life/',
 'https://theaggie.org/2018/03/06/duke-law-professor-presents-lecture-voting-rights-law/',
 'https://theaggie.org/2018/03/05/last-week-senate-25-8/',
 'https://theaggie.org/2018/03/05/icc-warns-students-wary-suspicious-job-offering/',
 'https://theaggie.org/2018/03/04/uc-davis-student-protester-arrested-yiannopoulos-talk-last-year-awaits-trial/',
 'https://theaggie.org/2018/03/01/executive-ticket-candidates-claim-biases-non-asucd-affiliated-individuals/',
 'https://theaggie.org/2018/03/01/happiness-campaign-strives-less-stress-joy/',
 'https://theaggie.org/2018/02/27/senator-term-review-yajaira-ramirez-sigala/',
 'https://theaggie.org/2018/02/27/calpirgs-winter-quar

In [4]:
print(len(Articles_url("https://theaggie.org/sports/",2)))
Articles_url("https://theaggie.org/sports/",2)

30


['https://theaggie.org/2018/03/11/doing-more-than-dribbling/',
 'https://theaggie.org/2018/03/09/uc-davis-wins-regular-season-big-west-conference-title-in-double-overtime-thriller/',
 'https://theaggie.org/2018/03/08/aggies-win-big-in-triple-overtime-victory/',
 'https://theaggie.org/2018/03/08/womens-basketball-tears-through-competition-in-2017-18-season/',
 'https://theaggie.org/2018/03/08/collaboration-brings-together-at-risk-youth-uc-davis-mens-basketball/',
 'https://theaggie.org/2018/03/06/lawsuit-filed-mlb-player-agency-performance-enhancing-drugs/',
 'https://theaggie.org/2018/03/04/proposed-california-bill-ban-tackle-football-children-14/',
 'https://theaggie.org/2018/03/01/womens-tennis-rolls-conference-play/',
 'https://theaggie.org/2018/03/01/womens-basketball-downs-uc-irvine-clinches-conference/',
 'https://theaggie.org/2018/03/01/shorts-buzzer-beater-wins-close-game-one-point/',
 'https://theaggie.org/2018/03/01/aggies-swept-creighton-heartbreaking-fashion/',
 'https://th

__Exercise 1.2.__ Write a function that extracts the title, text, and author of an Aggie article. The function should:

* Have a parameter `url` for the URL of the article.

* For the author, extract the "Written By" line that appears at the end of most articles. You don't have to extract the author's name from this line.

* Return a dictionary with keys "url", "title", "text", and "author". The values for these should be the article url, title, text, and author, respectively.

For example, for [this article](https://theaggie.org/2017/02/14/project-toto-aims-to-address-questions-regarding-city-finances/) your function should return something similar to this:
```
{
    'author': u'Written By: Bianca Antunez \xa0\u2014\xa0city@theaggie.org',
    'text': u'Davis residents create financial model to make city\'s financial state more transparent To increase transparency between the city\'s financial situation and the community, three residents created a model called Project Toto which aims to improve how the city communicates its finances in an easily accessible design. Jeff Miller and Matt Williams, who are members of Davis\' Finance and Budget Commission, joined together with Davis entrepreneur Bob Fung to create the model plan to bring the project to the Finance and Budget Commission in February, according to Kelly Stachowicz, assistant city manager. "City staff appreciate the efforts that have gone into this, and the interest in trying to look at the city\'s potential financial position over the long term," Stachowicz said in an email interview. "We all have a shared goal to plan for a sound fiscal future with few surprises. We believe the Project Toto effort will mesh well with our other efforts as we build the budget for the next fiscal year and beyond." Project Toto complements the city\'s effort to amplify the transparency of city decisions to community members. The aim is to increase the understanding about the city\'s financial situation and make the information more accessible and easier to understand. The project is mostly a tool for public education, but can also make predictions about potential decisions regarding the city\'s financial future. Once completed, the program will allow residents to manipulate variables to see their eventual consequences, such as tax increases or extensions and proposed developments "This really isn\'t a budget, it is a forecast to see the intervention of these decisions," Williams said in an interview with The Davis Enterprise. "What happens if we extend the sales tax? What does it do given the other numbers that are in?" Project Toto enables users, whether it be a curious Davis resident, a concerned community member or a city leader, with the ability to project city finances with differing variables. The online program consists of the 400-page city budget for the 2016-2017 fiscal year, the previous budget, staff reports and consultant analyses. All of the documents are cited and accessible to the public within Project Toto. "It\'s a model that very easily lends itself to visual representation," Mayor Robb Davis said. "You can see the impacts of decisions the council makes on the fiscal health of the city." Complementary to this program, there is also a more advanced version of the model with more in-depth analyses of the city\'s finances. However, for an easy-to-understand, simplistic overview, Project Toto should be enough to help residents comprehend Davis finances. There is still more to do on the project, but its creators are hard at work trying to finalize it before the 2017-2018 fiscal year budget. "It\'s something I have been very much supportive of," Davis said. "Transparency is not just something that I have been supportive of but something we have stated as a city council objective [ ] this fits very well with our attempt to inform the public of our challenges with our fiscal situation." ',
    'title': 'Project Toto aims to address questions regarding city finances',
    'url': 'https://theaggie.org/2017/02/14/project-toto-aims-to-address-questions-regarding-city-finances/'
}
```

Hints:

* The author line is always the last line of the last paragraph.

* If you are having trouble working with unicode characters see [the python 3 documentation](https://docs.python.org/3/howto/unicode.html).

In [5]:
def Articles(url):
    """
    The function is to extracts all of the content of a article
    
    Arg:
    url:the article url.
    """
    response = requests.get(url)
    doc = response.text
    
    html = lx.fromstring(doc, base_url = url)
    html.make_links_absolute()
    
    title = html.xpath("//h1[@itemprop = 'headline']")[0].text_content()
    text = "".join([x.text_content() for x in html.xpath("//div[@itemprop = 'articleBody']/p")[:-1]])
    author = html.xpath("//div[@itemprop = 'articleBody']/p[last()]")[0].text_content()
    
    return {"url":url,"title":title, "text":text,"author":author}

In [6]:
Articles("https://theaggie.org/2017/02/14/project-toto-aims-to-address-questions-regarding-city-finances/")

{'author': 'Written By: Bianca Antunez \xa0—\xa0city@theaggie.org',
 'text': 'Davis residents create financial model to make city’s financial state more transparentTo increase transparency between the city’s financial situation and the community, three residents created a model called Project Toto which aims to improve how the city communicates its finances in an easily accessible design. Jeff Miller and Matt Williams, who are members of Davis’ Finance and Budget Commission, joined together with Davis entrepreneur Bob Fung to create the model plan to bring the project to the Finance and Budget Commission in February, according to Kelly Stachowicz, assistant city manager. “City staff appreciate the efforts that have gone into this, and the interest in trying to look at the city’s potential financial position over the long term,” Stachowicz said in an email interview. “We all have a shared goal to plan for a sound fiscal future with few surprises. We believe the Project Toto effort will 

__Exercise 1.3.__ Use your functions from exercises 1.1 and 1.2 to get a data frame of 60 [Campus News](https://theaggie.org/campus/) articles and a data frame of 60 [City News](https://theaggie.org/city/) articles. Add a column to each that indicates the category, then combine them into one big data frame.

The "text" column of this data frame will be your corpus for natural language processing in exercise 1.4.

In [7]:
campus = [Articles(x) for x in Articles_url("https://theaggie.org/campus/",4)]
city = [Articles(x) for x in Articles_url("https://theaggie.org/city/",4)]

In [8]:
Campus=pd.DataFrame(campus)
Campus["category"] = "campus"
City = pd.DataFrame(city)
City["category"] = "city"
big = pd.concat([Campus,City])

In [9]:
big.head()

,author,text,title,url,category
0,Written by: George Liao — campus@theaggie.org,Funding for Student Advocates for Mental Healt...,Last week in Senate,https://theaggie.org/2018/03/13/last-week-in-s...,campus
1,Written by: Hannah Holzer — campus@theaggie.org,Proposed $978 tuition hikeAt the upcoming UC R...,UC Regents will vote on nonresident tuition hi...,https://theaggie.org/2018/03/12/uc-regents-wil...,campus
2,Written by: Sabrina Habchi — campus@theaggie.org,Controversy over academic honesty suspicions i...,Cheating allegations in SOC 46A,https://theaggie.org/2018/03/12/cheating-alleg...,campus
3,Written by: Jacqueline Moore — campus@theaggie...,"Task forces to review affordable housing, food...",Chancellor creates three task forces to improv...,https://theaggie.org/2018/03/12/chancellor-cre...,campus
4,Written by: Jacqueline Moore — campus@theaggie...,Guy-Uriel Charles speaks at King HallDuke Univ...,Duke law professor presents lecture on voting ...,https://theaggie.org/2018/03/06/duke-law-profe...,campus


In [10]:
big.tail()

,author,text,title,url,category
55,Written by: Ahash Francis — city@theaggie.org,Planning committees address Davis housing cris...,Social Services Commission meets to discuss We...,https://theaggie.org/2017/12/10/social-service...,city
56,,Anti-Monsanto protesters seek discovery from Y...,Monsanto protesters allege violation of rights,https://theaggie.org/2017/12/10/monsanto-prote...,city
57,Written by: Olivia Rockeman — city@theaggie.org,"Social Bikes, SACOG, cities of Davis, Sacramen...","Bike-share system to launch in Sacramento, Dav...",https://theaggie.org/2017/12/07/bike-share-sys...,city
58,Written by: Dylan Svoboda — city@theaggie.org,Restoration will give rise to ecological activ...,North Davis lot to go through habitat restoration,https://theaggie.org/2017/12/05/north-davis-lo...,city
59,“Ongoing issue with chickens being within 40 f...,Who let the chickens out?Nov. 15“Reporting par...,Police Logs,https://theaggie.org/2017/12/05/police-logs-34/,city


__Exercise 1.4.__ Use the Aggie corpus to answer the following questions. Use plots to support your analysis.

* What topics does the Aggie cover the most? Do city articles typically cover different topics than campus articles?

* What are the titles of the top 3 pairs of most similar articles? Examine each pair of articles. What words do they have in common?

* Do you think this corpus is representative of the Aggie? Why or why not? What kinds of inference can this corpus support? Explain your reasoning.

Hints:

*   The [nltk book](http://www.nltk.org/book/) and [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) may be helpful here.

*   You can determine whether city articles are "near" campus articles from the similarity matrix or with k-nearest neighbors.

*   If you want, you can use the [wordcloud](http://amueller.github.io/word_cloud/) package to plot a word cloud. To install the package, run
    ```
    conda install -c https://conda.anaconda.org/amueller wordcloud
    ```
    in a terminal. Word clouds look nice and are easy to read, but are less precise than bar plots.

In [11]:
import collections
import nltk
#nltk.download()
from textblob import TextBlob
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

from sklearn.feature_extraction.text import TfidfVectorizer


stopwords.extend(['uc',"davis","”","“","’","—"])

In [12]:
from nltk.corpus import wordnet

def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN)

In [13]:
texts = []
text2 = []
text_counter=collections.Counter()

for text in big.text:
    blob = TextBlob(text)
    new_text = " ".join(w for w in blob.words if w.lower() not in stopwords)
    blob = TextBlob(new_text)
    tags = [wordnet_pos(x[1]) for x in blob.pos_tags]
    new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
    blob = TextBlob(new_text)
    
    texts.append(new_text)
    text2.append(blob)
    text_counter.update(blob.word_counts)


In [14]:
text_counter.most_common(6)

[('say', 760),
 ('student', 750),
 ('city', 254),
 ('community', 253),
 ('people', 240),
 ('make', 225)]

So that, student, city ,comunity is the three topics the Aggie cover the most if 'say' is not a meaningful topic.

In [15]:
citytext = collections.Counter()
for text in big.text[:60]:
    blob = TextBlob(text)
    new_text = " ".join(w for w in blob.words if w.lower() not in stopwords)
    blob = TextBlob(new_text)
    tags = [wordnet_pos(x[1]) for x in blob.pos_tags]
    new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
    blob = TextBlob(new_text)
    
    citytext.update(blob.word_counts)

In [16]:
campustext = collections.Counter()
for text in big.text[-60:]:
    blob = TextBlob(text)
    new_text = " ".join(w for w in blob.words if w.lower() not in stopwords)
    blob = TextBlob(new_text)
    tags = [wordnet_pos(x[1]) for x in blob.pos_tags]
    new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
    blob = TextBlob(new_text)
    
    campustext.update(blob.word_counts)

The more words they have in common, the larger the dot product will be.

In [17]:
df = pd.DataFrame([citytext,campustext]).T.fillna(0)
tf = df/df.sum()
sum(tf[0]*tf[1])

0.0010381843900213136

the value is so small. It may be different.

### What are the titles of the top 3 pairs of most similar articles? Examine each pair of articles. What words do they have in common?

In [18]:
# to get the frequcy of the words of the passage
def fun1(number):
    count = collections.Counter()
    text = big.text.iloc[number]
    blob = TextBlob(text)
    new_text = " ".join(w for w in blob.words if w.lower() not in stopwords)
    blob = TextBlob(new_text)
    tags = [wordnet_pos(x[1]) for x in blob.pos_tags]
    new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
    blob = TextBlob(new_text)
    count.update(blob.word_counts)
    return count

In [19]:
#get the top 10 words most in common
def fun2(number1,number2):
    a=fun1(number1)
    b=fun1(number2)
    df = pd.DataFrame([a,b]).T.fillna(0)
    tf = df/df.sum()
    results = (tf[0]*tf[1]).sort_values(ascending=False)[:10].index
    return list(results)

In [20]:
import numpy as np
tf_idf = TfidfVectorizer().fit_transform(texts)
similarity = (tf_idf * tf_idf.T-np.eye(120))

The first pair

In [21]:
np.unravel_index(similarity.argmax(), similarity.shape)

(8, 20)

In [22]:
big.title.iloc[8]

'Executive ticket candidates claim biases against non-ASUCD affiliated individuals'

In [23]:
big.title.iloc[20]

'Adam Hatefi and Shreya Deshpande disqualified after missing mandatory Candidate Workshop'

In [24]:
fun2(8,20)

['committee',
 'candidate',
 'elections',
 'say',
 'deshpande',
 'asucd',
 'workshop',
 'slate',
 'disqualification',
 'hatefi']

The second pair

In [25]:
similarity[8,20]=0
similarity[20,8]=0
np.unravel_index(similarity.argmax(), similarity.shape)

(18, 43)

In [26]:
big.title.iloc[18]

'Mental Health Town Hall erupts in protest'

In [27]:
big.title.iloc[43]

'Internal audit reveals steep decline in counselors, $250,000 worth of questionably allocated funds'

In [28]:
fun2(18,43)

['student',
 'say',
 'health',
 'counselor',
 'mental',
 'walter',
 'audit',
 'staff',
 'service',
 'need']

the third pair

In [29]:
similarity[18,43]=0
similarity[43,18]=0
np.unravel_index(similarity.argmax(), similarity.shape)

(17, 25)

In [30]:
big.title.iloc[17]

'ASUCD executive tickets hold CoHo debate'

In [31]:
big.title.iloc[25]

'2018 ASUCD Winter Elections — meet the candidates'

In [32]:
fun2(17,25)

['student',
 'asucd',
 'run',
 'major',
 'senate',
 'slate',
 'transfer',
 'third-year',
 'work',
 'campus']

The last part.

In [33]:
text_counter.most_common(20)

[('say', 760),
 ('student', 750),
 ('city', 254),
 ('community', 253),
 ('people', 240),
 ('make', 225),
 ('work', 224),
 ('also', 214),
 ('campus', 193),
 ('would', 187),
 ('health', 185),
 ('year', 182),
 ('go', 182),
 ('one', 162),
 ('need', 158),
 ('member', 156),
 ('get', 154),
 ('project', 152),
 ('major', 149),
 ('asucd', 146)]

Most of the word above is about the school activity or the subject near school.  I think is representative of the Aggie.